In [1]:
import numpy as np
import tensorflow as tf
import utils
import word2vec_utils
from tensorflow.contrib.tensorboard.plugins import projector

define Model hyperparameters

In [2]:
VOCAB_SIZE = 50000
BATCH_SIZE = 128
EMBED_SIZE = 500
SKIP_WINDOW = 2
NUM_SAMPLED  = 64
LEARNING_RATE = 1.0
NUM_TRAN_STEPS = 10000
VISUAL_FLD = 'visualization'
SKIP_STEP = 5000

In [4]:
DOWNLOAD_URL = 'http://mattmahoney.net/dc/text8.zip'
EXPECTED_BYTES = 31344016
NUM_VISUALIZE = 3000        # number of tokens to visualize
def word2vec(dataset):
    with tf.name_scope('data'):
        iterator = dataset.make_initializable_iterator()
        center_words,target_words = iterator.get_next()
    with tf.name_scope('embed'):
        embed_matrix = tf.get_variable('embed_matrix',
                                      shape = [VOCAB_SIZE,EMBED_SIZE],
                                      initializer = tf.random_uniform_initializer())
        embed = tf.nn.embedding_lookup(embed_matrix,center_words,name = 'embedding')
    with tf.name_scope('loss'):
        nce_weight = tf.get_variable('nce_weight',shape = [VOCAB_SIZE,EMBED_SIZE],
                                    initializer = tf.truncated_normal_initializer(stddev = 1.0/(EMBED_SIZE**0.5)))
        nce_bias = tf.get_variable('nce_bias',initializer = tf.zeros([VOCAB_SIZE]))
        loss = tf.reduce_mean(tf.nn.nce_loss(weights = nce_weight,
                                            biases = nce_bias,
                                            labels = target_words,
                                            inputs = embed,
                                            num_sampled = NUM_SAMPLED,
                                            num_classed = VOCAB_SIZE),name = 'loss')
    with tf.name_scope('optimizer'):
        optimizer = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(loss)
    
    utils.safe_mkdir('checkpoints')
    
    with tf.Session() as sess:
        sess.run(iterator.initializer)
        sess.run(tf.global_variables_initializer)
        total_loss = 0.0
        writer = tf.summary.FileWriter('graphs/word2vec_simple',sess.graph)
        for i in range(NUM_TRAIN_STEPS):
            try:
                loss_batch,_ = sess.run([loss,optimizer])
                total_loss +=loss_batch
                if (index + 1) % SKIP_STEP == 0:
                    print ('Average loss at step {}: {:5.1f}'.format(index,total_loss/SKIP_STEP))
                    total_loss = 0.0
            except tf.errors.OutOfRangeError:
                sess.run(iterator.initializer)
    return 
def gen():
    yield from word2vec_utils.batch_gen(DOWNLOAD_URL,
                                        EXPECTED_BYTES,
                                        VOCAB_SIZE,
                                        BATCH_SIZE,
                                        SKIP_WINDOW,
                                       VISUAL_FLD)
    return
def main():
    dataset = tf.data.Dataset.from_generator(gen,
                                             (tf.int32,tf.int32),
                                             (tf.TensorShape([BATCH_SIZE]),tf.TensorShape([BATCH_SIZE,1]))
                                            )
    return
if __name__ == '__main__':
    main()